# Титаник: Выбор модели

Рассмотрим данные о пассажирах с печально известного Титаника. У нас есть неполные данные о 890 пассажирах. Составим несколько моделей выживания и сравним их точность. Все шаги можно посмотреть по ссылке выше или в блокноте Титаник: Подготовка данных.

Основная часть блокнота со всеми комментариями доступна [здесь](https://www.kaggle.com/startupsci/titanic-data-science-solutions/notebook).
Также мы уже обработали данные и убрали из них все вещественные и строковые значения.

Импортируем основные модули.

In [32]:
# анализ данных
import pandas as pd
import numpy as np
import random as rnd

# машинное обучение
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn import metrics

Загрузим подготовленные данные из файлов.

In [33]:
train_df = pd.read_csv('train_upd.csv')
test_df = pd.read_csv('test_upd.csv')

In [34]:
# изучим данные
train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1,0,0,1,0,3
1,1,1,1,2,3,1,3,0,2
2,1,3,1,1,1,0,2,1,3
3,1,1,1,2,3,0,3,0,2
4,0,3,0,2,1,0,1,1,6


In [35]:
test_df.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,892,3,0,2,0,2,1,1,6
1,893,3,1,2,0,0,3,0,6
2,894,2,0,3,1,2,1,1,6
3,895,3,0,1,1,0,1,1,3
4,896,3,1,1,1,0,3,0,3


### Базовая модель (Baseline)

Под базовой моделью (baseline) подразумевается какая-то несложная модель или константное предсказание, от которого мы не ждем суперкачества, но на которую мы ориентируемся впоследствии (стараемся предсказать лучше, чем она). 

Самая простая базовая модель - константное предсказание. Мы просто берем какое-то значение и используем его (значение одного из классов для всех наблюдений для задачи классификации или среднее значение y для задач регрессии). 

### Accuracy

Accuracy представляет собой простую метрику для задач классификации, которая считает количество верно угаданных предсказаний относительно общего количество элементов. Например, у нас есть y = [0 1 0 0 1 0 1 1]. Мы предсказали y_pred = [0 1 0 1 1 0 0 1]. 6 из 8 правильных, получим accuracy score = 0.75.

Также для задач классификации можно использовать метрику Presicion, F1-score или Log-loss. С ними можно ознакомится по [ссылке](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics).

Для задач регрессии можно использовать следующие метрики:

### Root Mean Squared Error (RMSE)

Root Mean Squared Error или среднеквадратичное отклонение. Одна из основных метрик для задач регрессии, которая при этом является относительно простой. Она считается по следующей формуле:<br>
![alternate text](https://wikimedia.org/api/rest_v1/media/math/render/svg/15619dfbb9a470d310c1bc08fb61d4fb1187d057)

Т.е. представляет собой квадратный корень из разности квадратов предсказанного и реального значений.

### Mean Absolute Error (MAE)

В MAE ошибка считается как среднее от абсолютной разницы между реальным и предсказанным значений. MAE это линейная оценка, что означает что все индивидуальные разницы в среднем взвешены одинаково. Например, разница между 10 и 0 будет двойной разницей между 5 и 0. Что не будет верно в случае RMSE. Математически формулы высчитывается вот так:<br> 
![alternate text](https://cdn-images-1.medium.com/max/1600/1*8DXbECB9pnKxTpIvuVD-vg.png)


Почитать подробнее про метрики для регрессии и базовую модель можно по этой [ссылке](https://towardsdatascience.com/how-to-select-the-right-evaluation-metric-for-machine-learning-models-part-1-regrression-metrics-3606e25beae0).

Так как у нас задача классификации, то будем использовать метрику аccuracy.


### Submission

Попробуем загрузить на Kaggle первый сабмишн и одновременно создать константную базовую модель. Если изучить данные, то видно что только 38% всех пассажиров из обучающей выборки выжили. Если такая же пропорция сохранится для тестовой выборки, то можно предположить что большинство людей умерли в тестовой выборке. Поэтому сделаем констатное предсказание, что `ВСЕ` пассажиры умерли и сохраним его в csv файл для дальнейшей загрузки на Kaggle.

In [75]:
# Заполняем нулями список выживших, точнее уже погибших
Y_pred = list(0 for i in range(len(test_df)))

submission_dead = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })

submission_dead.to_csv('submission_dead.csv', index=False)

Сразу сделаем второй сабмишн. Изучив данные о тех кто выжил, можно заметить что в основном выжили женщины:

In [41]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
1,1,0.742038
0,0,0.188908


Мы закодировали женский пол как 1 и мужской как 0 (в блокноте Титаник: Подготовка данных). Почти 75% женщин выжили, что хорошо соотносится с фразой "сначала женщины и дети".

Поэтому можно сделать новое предсказание, что погибли все, кроме женщин.

In [74]:
Y_pred = list(test_df.Sex[i] for i in range(len(test_df)))

submission_woman = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })

submission_woman.to_csv('submission_woman.csv', index=False)

Предсказания можно загрузить на Kaggle. Предсказание, где женщины выжили, должно набрать больше очков. Т.е. мы улучшим свое предсказание относительно базовой модели. Сделать их можно по следующему [адресу](https://www.kaggle.com/c/titanic/submit). Наше score 0.62679 для случая когда все погибли. А score для случая когда все женщины выжили: 0.76555. Улучшили результат на 0.13876 относительно базавой модели.

### Model, predict и solve

Когда мы обработали весь дасет, убрав из него все вещественные и строковые значения, можно заняться обучением модели и предсказанием выживания пассажиров. Для начала выберем алгоритм предсказания. Сейчас существует боллее 60 алгоритмов. Чтобы выбрать правильный, нужно понимать тип задачи и решение, которое удовлетворит нашим условиям. У нас задача классификации и регрессии. Нужно понять соотношение между результатом (выжил или нет) и другими переменными или признаками (пол, возраст, порт, класс и т.д.). По факту мы осуществляем обучение с учителем, т.к. обучение идет с известным результатом (выжил или нет). С этими параметрами, мы можем ограничить список подходящих нам алгоритмов. Остановимся в итоге на этих алгоритмах:

* Логистическая регрессия
* KNN или k-Ближайших соседей (k-Nearest Neighbors)
* Наивный Байес
* Дерево решений
* Случайный лес
* Перцептрон
* XGBoost

In [6]:
from sklearn.model_selection import train_test_split

# заполнили массивы с нашими данными 
X = train_df.drop("Survived", axis=1)
y = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

# Делим
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state = 0)

Посмотрим как наши данные разбились:

In [8]:
train_X.shape, train_y.shape, test_X.shape, test_y.shape, X_test.shape

((668, 8), (668,), (223, 8), (223,), (418, 8))

668 элементов в обучающей и 223 элемента в тестовой выборке.

Начнем с Логистической регрессии. Ее советуют использовать в самом начале. Эта статистическая модель, используемая для предсказания вероятности возникновения некоторого события путём подгонки данных к логистической кривой.

Давайте обучим модель на нашей обучающей выборке и предскажем людям из тестовой выборке выжили они или нет. А потом посмотрим насколько точно она это сделала.

In [13]:
# Логистическая регрессия

logreg = LogisticRegression()
logreg.fit(train_X, train_y)
pred = logreg.predict(test_X)

acc_log = round(metrics.accuracy_score(test_y, pred) * 100, 2)
acc_log

76.68

Сделаем первый сабмишн на Kaggle с Логистической регрессией. Сохраним файл как csv, а потом загрузим его на сайт, чтобы узнать насколько точно мы предсказали.

In [31]:
Y_pred = logreg.predict(X_test)

submission_LR = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })

submission_LR.to_csv('submission_LR.csv', index=False)

Продолжим выяснять какая модель лучше. На очереди KNN. 

Запустим алгоритм по трем ближайщим соседям.

In [14]:
# KNN

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_X, train_y)
pred = knn.predict(test_X)

acc_knn = round(metrics.accuracy_score(test_y, pred) * 100, 2)
acc_knn

81.61

Результат KNN уже лучше чем у Логистической регрессии. Попробуем теперь Наивный Байес.

In [15]:
# Наивный Байес

gaussian = GaussianNB()
gaussian.fit(train_X, train_y)
pred = gaussian.predict(test_X)

acc_gaussian = round(metrics.accuracy_score(test_y, pred) * 100, 2)
acc_gaussian

72.65

Хм. Результат ухудшился, но ничего. У нас есть еще несколько модель. Следующей будет Перцептрон. Это самая простейшая модель нейронной сети. 

Перцептрон - это алгоритм обучения с учителем двоичных классификаторов (функций, которые могут определять, принадлежит ли вход, представленный вектором чисел, какому-то определенному классу или нет). Это тип линейного классификатора, то есть алгоритм классификации, который делает свои предсказания на основе функции линейного предиктора, объединяющей набор весов с вектором признаков.

Дополнительно почитать про перцептроны можно [здесь](http://neuralnet.info/chapter/персептроны/).

In [16]:
# Перцептрон

perceptron = Perceptron()
perceptron.fit(train_X, train_y)
pred = perceptron.predict(test_X)

acc_perceptron = round(metrics.accuracy_score(test_y, pred) * 100, 2)
acc_perceptron

/Users/Mamont/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


74.89

Немного лучше. Попробуем Дерево решений.

Эта модель использует дерево решений в качестве прогнозирующей модели, которая соединяет признаки (ветви дерева) с выводами о принадлежности к какому-то из классов (листья дерева). Древовидные модели, в которых целевая переменная может принимать конечный набор значений, называются деревьями классификации; в этих древовидных структурах листья представляют метки классов, а ветви представляют соединения функций, которые ведут к этим меткам классов.

In [27]:
# Дерево решений

decision_tree = DecisionTreeClassifier()
decision_tree.fit(train_X, train_y)
pred = decision_tree.predict(test_X)

acc_decision_tree = round(metrics.accuracy_score(test_y, pred) * 100, 2)
acc_decision_tree

82.06

Самой популярной моделью является Случайный лес. Алгоритм заключается в использовании ансамбля решающих деревьев. Алгоритм сочетает в себе две основные идеи: метод бэггинга Бреймана, и метод случайных подпространств, предложенный Tin Kam Ho. Алгоритм применяется для задач классификации, регрессии и кластеризации. Основная идея заключается в использовании большого ансамбля решающих деревьев, каждое из которых само по себе даёт очень невысокое качество классификации, но за счёт их большого количества результат получается хорошим.

Укажем максимальное количество деревьев равным 100.

In [18]:
# Случайный Лес

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_X, train_y)
pred = random_forest.predict(test_X)

acc_random_forest = round(metrics.accuracy_score(test_y, pred) * 100, 2)
acc_random_forest

84.3

Последней моделью будет XGBoost -  библиотека, реализующая методы градиентного бустинга. Она работает очень похоже на Случайный лес, тоже создавая много неглубоких деревьев. Идея градиентного бустинга состоит в построении ансамбля последовательно уточняющих друг друга элементарных моделей. n-ная элементарная модель обучается на “ошибках” ансамбля из n-1 моделей, ответы моделей взвешенно суммируются. “Ошибки” здесь в кавычках, поскольку на самом деле каждая последующая модель приближает антиградиент функции потерь, который не обязательно равен разности фактических и предсказанных значений (т.е. ошибке в буквальном смысле). 

Чуть больше информации можно изучить по этой [ссылке](http://biostat-r.blogspot.com/2016/08/xgboost.html). Не пугайтесь что там R, теория и применение справедливо и для питона.

Мы будем использовать классификатор с 100 деревьями.

In [28]:
# XGBoost

xgb = XGBClassifier(n_estimators=100)
xgb.fit(train_X,train_y)
pred = xgb.predict(test_X)

acc_xgb = round(metrics.accuracy_score(test_y, pred) * 100, 2)
acc_xgb

/Users/Mamont/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


82.96

### Оценка моделей

Мы теперь можем оценить все модели и выбрать самую лучшую для нашей задачи. С небольшим отрывом побеждает Случайный лес. Его то мы и используем для финального сабмита на Kaggle.

In [29]:
models = pd.DataFrame({
    'Model': ['KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Decision Tree', 'XGBoost'],
    'Score': [acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron,
              acc_decision_tree, acc_xgb]})
models.sort_values(by='Score', ascending=False)

,Model,Score
2,Random Forest,84.30
6,XGBoost,82.96
5,Decision Tree,82.06
0,KNN,81.61
1,Logistic Regression,76.68
4,Perceptron,74.89
3,Naive Bayes,72.65


Сохраним в csv наш сабмишн.

In [30]:
Y_pred = random_forest.predict(X_test)

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })

submission.to_csv('submission_RF.csv', index=False)

Теперь у нас есть 4 сабмишена. Стоит напомнить, что в день можно совершать до 10 сабмишинов. Теперь можно загрузить их на сайт и проверить какие результаты у нас получились и отметить насколько улучшились наши предсказания относительно базовой модели. На Логистической регрессии получили 0.77511.